In [20]:
import pycountry

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 24, Finished, Available, Finished)

In [21]:
df = spark.read.parquet("Files/gold/earthquake_events_gold/")

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 25, Finished, Available, Finished)

In [ ]:
manual_overrides = {
    "PR": "Puerto Rico",
    "HK": "Hong Kong",  # sometimes omitted depending on pycountry version
    "MO": "Macau",
    "TW": "Taiwan",  # depending on version/config
    "XK": "Kosovo",
    "GG": "Guernsey",
    "JE": "Jersey",
    "IM": "Isle of Man",
    "PS": "Palestine",
    "SS": "South Sudan",  # sometimes missing in older pycountry
    "AQ": "Antarctica",  # rare
    "ZZ": "Unknown or Invalid"
}

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 26, Finished, Available, Finished)

In [23]:
# Define UDF
def get_country_name(code):
    if code is None:
        return None
    code = code.upper()
    if code in manual_overrides:
        return manual_overrides[code]
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.name if country else code
    except:
        return code

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 27, Finished, Available, Finished)

In [24]:
# Register as UDF
get_country_name_udf = udf(get_country_name, StringType())

# Apply UDF to Spark DataFrame
df = df.withColumn("country_name", get_country_name_udf(df["country_code"]))

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 28, Finished, Available, Finished)

In [25]:
df_clean = df.dropDuplicates()

StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 29, Finished, Available, Finished)

In [27]:
df_clean.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("Cleaned_earthquake_table")


StatementMeta(, 1412a0d9-5808-4047-b8cc-d4fdf0c3c809, 31, Finished, Available, Finished)